In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import glob 
import matplotlib.colors as mcolors
import os 
#import librosa 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import IPython.display as ipd
from IPython.display import SVG, display
import random
import math
import sklearn
import shutil  
import scipy 
from sklearn.cluster import KMeans
import matplotlib.style
from itertools import compress 
import matplotlib as mpl
mpl.style.use('ggplot')
couleurs = mcolors.CSS4_COLORS
Tcolors = list(couleurs.keys())

In [3]:
#!pip install librosa

In [4]:
def create_directory(path = "/tmp/year") :
    try:
        if not os.path.exists(path):
            os.makedirs(path)
            print("Directory " , path ,  " Created ")
        else:    
            print("Directory " , path ,  " already exists")
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s" % path)
        
def remove_directory(path = "/tmp/year") :
    try:
        os.rmdir(path)
    except OSError:
        print ("Deletion of the directory %s failed" % path)
    else:
        print ("Successfully deleted the directory %s" % path)
        
        
def copy_file(file = "txt.txt", destination = "/tmp/year") :

    # Copy the file to destination 
    try: 
        shutil.copy(file, destination) 
        print("File copied successfully.") 
    # If source and destination are same 
    except shutil.SameFileError: 
        print("Source and destination represents the same file.") 
    # If destination is a directory. 
    except IsADirectoryError: 
        print("Destination is a directory.") 
    # If there is any permission issue 
    except PermissionError: 
        print("Permission denied.") 
    # For other errors 
    except: 
        print("Error occurred while copying file.") 

In [5]:
def read_songs_data() :
    all_data = ['genres/blues/*', 'genres/classical/*', 'genres/country/*', 'genres/disco/*', 'genres/hiphop/*', 'genres/jazz/*', 'genres/metal/*',  'genres/pop/*',  'genres/reggae/*', 'genres/rock/*']
    all_the_songs_data = glob.glob(all_data[0]) + glob.glob(all_data[1]) + glob.glob(all_data[2]) + glob.glob(all_data[3]) + glob.glob(all_data[4]) + glob.glob(all_data[5]) + glob.glob(all_data[6]) + glob.glob(all_data[7]) + glob.glob(all_data[8]) + glob.glob(all_data[9]) 
    assert len(all_the_songs_data) == 1000
    s = []
    for x in all_the_songs_data :
        sx = x.replace('genres/', '')
        sxx = sx.replace('\\', '/')
        sxx = sxx[sxx.find('/')+1:]
        s.append(sxx)
    
    return all_the_songs_data, s, all_data

In [6]:
def get_name_wav(file_at) :
    #file_at = files_index[10]
    index_begin = file_at.find("\\")
    index_end = file_at.find(".wav")
    file_at = file_at[index_begin + 1 : index_end]
    return file_at

def get_name(file_at) :
    index_begin = file_at.find("\\")
    file_at = file_at[index_begin + 1 : ]
    return file_at

def get_speaker_id(file_at) :
    temp = file_at.split("_")
    return temp[0]

In [7]:
def get_dictionary_by_occurence(current, numUtterances = 4) :
    #print("Current file : ", current)
    current_files = glob.glob(current)
    temp_names = [get_name_wav(x) for x in current_files]
    speakers_id = [get_speaker_id(x) for x in temp_names]
    
    lookup_table = {}
    for index, element in enumerate(speakers_id) :
        if element in lookup_table :
            lookup_table[element].append(current_files[index])
        else :
            lookup_table[element] = [current_files[index]]
    
    
    temp_names = [get_name_wav(x) for x in current_files]
    speakers_id = [get_speaker_id(x) for x in temp_names]
    #print("len(speakers_id) = ", len(speakers_id))

    dictionary = {}
    for key in speakers_id : 
        if key in dictionary :
            dictionary[key] = dictionary[key] + 1
        else :
            dictionary[key] = 1

    newDict = dict()
    # Iterate over all the items in dictionary and filter items which has even keys
    for (key, value) in dictionary.items():
        # Check if key is even then add pair to new dictionary
        if value >= numUtterances:
            newDict[key] = value
    #print('Filtered Dictionary : ')
    
    return newDict, lookup_table

## Create data for speakers with a certain number of utterances

In [8]:
all_the_songs_data = glob.glob("speech_commands_data/*")

In [9]:
all_the_songs_data

['speech_commands_data\\bed',
 'speech_commands_data\\bird',
 'speech_commands_data\\cat',
 'speech_commands_data\\dog',
 'speech_commands_data\\down',
 'speech_commands_data\\eight',
 'speech_commands_data\\five',
 'speech_commands_data\\four',
 'speech_commands_data\\go',
 'speech_commands_data\\happy',
 'speech_commands_data\\house',
 'speech_commands_data\\left',
 'speech_commands_data\\marvin',
 'speech_commands_data\\nine',
 'speech_commands_data\\no',
 'speech_commands_data\\off',
 'speech_commands_data\\on',
 'speech_commands_data\\one',
 'speech_commands_data\\right',
 'speech_commands_data\\seven',
 'speech_commands_data\\sheila',
 'speech_commands_data\\six',
 'speech_commands_data\\stop',
 'speech_commands_data\\three',
 'speech_commands_data\\tree',
 'speech_commands_data\\two',
 'speech_commands_data\\up',
 'speech_commands_data\\wow',
 'speech_commands_data\\yes',
 'speech_commands_data\\zero']

In [10]:
all_files = ["speech_commands_data/" + get_name(x) + "/*" for x in all_the_songs_data]

In [11]:
print(len(all_files))
print(all_files)

30
['speech_commands_data/bed/*', 'speech_commands_data/bird/*', 'speech_commands_data/cat/*', 'speech_commands_data/dog/*', 'speech_commands_data/down/*', 'speech_commands_data/eight/*', 'speech_commands_data/five/*', 'speech_commands_data/four/*', 'speech_commands_data/go/*', 'speech_commands_data/happy/*', 'speech_commands_data/house/*', 'speech_commands_data/left/*', 'speech_commands_data/marvin/*', 'speech_commands_data/nine/*', 'speech_commands_data/no/*', 'speech_commands_data/off/*', 'speech_commands_data/on/*', 'speech_commands_data/one/*', 'speech_commands_data/right/*', 'speech_commands_data/seven/*', 'speech_commands_data/sheila/*', 'speech_commands_data/six/*', 'speech_commands_data/stop/*', 'speech_commands_data/three/*', 'speech_commands_data/tree/*', 'speech_commands_data/two/*', 'speech_commands_data/up/*', 'speech_commands_data/wow/*', 'speech_commands_data/yes/*', 'speech_commands_data/zero/*']


In [12]:
def get_utterances_and_speakers_to_include(from_folder, to_folder , numUtterances = 5, cutOff = 100, applyCutOff = False):
    all_data = glob.glob(from_folder)
    all_files = ["speech_commands_data/" + get_name(x) + "/*" for x in all_data]
    dirToCreate = [to_folder + get_name(x) for x in all_data]
    
    # Get the numbers of speakers in each folders having #numUtterances (5) utterances of the same words 
    numSpeakersWithOcurrences = []
    for x in all_files :
        dict_current, _= get_dictionary_by_occurence(x, numUtterances) 
        #print(len(dict_current))
        numSpeakersWithOcurrences.append(len(dict_current))
    print(numSpeakersWithOcurrences)
    
    # Get the folders where the numbers of spearker having #numUtterances (5) utterances >= cutOff
    index_cutOff = [False]*len(numSpeakersWithOcurrences)
    for index, x  in enumerate(numSpeakersWithOcurrences) :
        if x >= cutOff :
            index_cutOff[index] = True
    len_cutOff = sum(index_cutOff)
    
    
    if len_cutOff > 2 and applyCutOff :
        all_files = list(compress(all_files, index_cutOff)) 
        dirToCreate = list(compress(dirToCreate, index_cutOff)) 
        numSpeakersWithOcurrences = list(compress(numSpeakersWithOcurrences, index_cutOff)) 
    
    # Find the keys of the same speakers having 5 utterances of the same words accross differents
    dict_zero,_ = get_dictionary_by_occurence(all_files[0], numUtterances) 
    z = set(dict_zero.keys())
    for x in all_files[1:] :
        dict_current,_ = get_dictionary_by_occurence(x, numUtterances) 
        u = set(dict_current.keys())
        z = u.intersection(z)

    print("#spearkers intersected : ", len(z))
    final_speakers = z
    final_speakers_map = {}
    for index, element in enumerate(final_speakers) :
        final_speakers_map[element] = index
    
    return all_files, dirToCreate, numSpeakersWithOcurrences, final_speakers, final_speakers_map

In [13]:
def same_speakers_accross_folders(all_files, dirToCreate, numUtterances, final_speakers, final_speakers_map) : 
    for index, path in enumerate(dirToCreate) :
        current = all_files[index]
        print("Directory : ", current)
        dict_current, lookup_table = get_dictionary_by_occurence(current, numUtterances)
        speakers_identifiers = list(dict_current.keys())
        numSpeakers = len(speakers_identifiers)
        # Create the directory with commands title 
        create_directory(path) 

        for speakerIndex, IDs in enumerate(speakers_identifiers) :
            if IDs in final_speakers : 
                temp_files = lookup_table[IDs]
                # Create directory 
                newPath = path + "/speaker_" + str(final_speakers_map[IDs] + 1)
                create_directory(newPath) 
                for individual_file in temp_files :
                    # Copy all files to newPath 
                    copy_file(file = individual_file, destination = newPath) 
                #print(temp_files)
        print("Done dir_", index)
    print("Word Done")

In [14]:
def different_speakers_accross_folders(all_files, dirToCreate, numUtterances, minSpeakerPerFolder) :  
    for index, path in enumerate(dirToCreate) :
        current = all_files[index]
        dict_current, lookup_table = get_dictionary_by_occurence(current, numUtterances)
        speakers_identifiers = list(dict_current.keys())
        numSpeakers = len(speakers_identifiers)

        if numSpeakers >= minSpeakerPerFolder :
            # Create the directory with commands title 
            create_directory(path) 
        else :
            continue 

        for speakerIndex, IDs in enumerate(speakers_identifiers) :
            temp_files = lookup_table[IDs]
            # Create directory 
            newPath = path + "/speaker_" + str(speakerIndex + 1)
            create_directory(newPath) 
            for individual_file in temp_files :
                # Copy all files to newPath 
                copy_file(file = individual_file, destination = newPath) 
            #print(temp_files)
        print("Done dir_", index)
    print("Word Done")
    #print(speakers_identifiers)

### Create data with potentially different speakers accross folders

In [15]:
from_folder = "speech_commands_data/*"
to_folder = "speaker_data_data_diff/"
numUtterances = 5
cutOff = 100
applyCutOff = False
#                                        (from_folder, to_folder , numUtterances = 5, cutOff = 100, applyCutOff = False)
return_values = get_utterances_and_speakers_to_include(from_folder, to_folder , numUtterances, cutOff, applyCutOff)
all_files, dirToCreate, numSpeakersMat, final_speakers, final_speakers_map = return_values

[4, 1, 2, 4, 149, 150, 149, 148, 152, 2, 6, 149, 1, 155, 150, 152, 150, 154, 150, 151, 8, 153, 151, 149, 7, 150, 153, 3, 154, 155]
#spearkers intersected :  0


In [16]:
numUtterances = 6  # 6 repetitions for each speaker
minSpeakerPerFolder = 5 # Minimun number of speakers in each folders, diff speakers 
different_speakers_accross_folders(all_files, dirToCreate, numUtterances, minSpeakerPerFolder)

Directory  speaker_data_data_diff/down  Created 
Successfully created the directory speaker_data_data_diff/down
Directory  speaker_data_data_diff/down/speaker_1  Created 
Successfully created the directory speaker_data_data_diff/down/speaker_1
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/down/speaker_2  Created 
Successfully created the directory speaker_data_data_diff/down/speaker_2
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/down/speaker_3  Created 
Successfully created the directory speaker_data_data_diff/down/speaker_3
File copied successfully.
File copied successfully

File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/nine/speaker_5  Created 
Successfully created the directory speaker_data_data_diff/nine/speaker_5
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/nine/speaker_6  Created 
Successfully created the directory speaker_data_data_diff/nine/speaker_6
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/nine/speaker_7  Created 
Successfully created the directory speaker_data_data_diff/nine/speaker_7
File copied successfully.
File c

File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/on/speaker_6  Created 
Successfully created the directory speaker_data_data_diff/on/speaker_6
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/on/speaker_7  Created 
Successfully created the directory speaker_data_data_diff/on/speaker_7
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/on/speaker_8  Created 
Successfully created the directory speaker_data_data_diff/on/speaker_8
File copied successfully.
File copied successfully.
File copied successfully.
File copied succes

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/seven/speaker_7  Created 
Successfully created the directory speaker_data_data_diff/seven/speaker_7
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Done dir_ 19
Directory  speaker_data_data_diff/six  Created 
Successfully created the directory speaker_data_data_diff/six
Directory  speaker_data_data_diff/six/speaker_1  Created 
Successfully created the directory speaker_data_data_diff/six/speaker_1
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/six/speaker_2  Created 
Successfully created the directory speaker_data_data_diff/six/speaker_2
File copied suc

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/two/speaker_3  Created 
Successfully created the directory speaker_data_data_diff/two/speaker_3
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/two/speaker_4  Created 
Successfully created the directory speaker_data_data_diff/two/speaker_4
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/two/

File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/zero/speaker_5  Created 
Successfully created the directory speaker_data_data_diff/zero/speaker_5
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/zero/speaker_6  Created 
Successfully created the directory speaker_data_data_diff/zero/speaker_6
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_diff/zero/speaker_7  Created 
Successfully created the directory speaker_data_data_diff/zero/speaker_7
File copied successfully.
File copied successfully.
File c

### Create data with same speakers accross folders

In [17]:
from_folder = "speech_commands_data/*"
to_folder = "speaker_data_data_same/"
numUtterances = 5
cutOff = 100
applyCutOff = True
#                                        (from_folder, to_folder , numUtterances = 5, cutOff = 100, applyCutOff = False)
return_values = get_utterances_and_speakers_to_include(from_folder, to_folder , numUtterances, cutOff, applyCutOff)
all_files, dirToCreate, numSpeakersMat, final_speakers, final_speakers_map = return_values

[4, 1, 2, 4, 149, 150, 149, 148, 152, 2, 6, 149, 1, 155, 150, 152, 150, 154, 150, 151, 8, 153, 151, 149, 7, 150, 153, 3, 154, 155]
#spearkers intersected :  122


In [18]:
len(dirToCreate)

20

In [19]:
same_speakers_accross_folders(all_files, dirToCreate, numUtterances, final_speakers, final_speakers_map)

Directory :  speech_commands_data/down/*
Directory  speaker_data_data_same/down  Created 
Successfully created the directory speaker_data_data_same/down
Directory  speaker_data_data_same/down/speaker_28  Created 
Successfully created the directory speaker_data_data_same/down/speaker_28
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_109  Created 
Successfully created the directory speaker_data_data_same/down/speaker_109
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_57  Created 
Successfully created the directory speaker_data_data_same/down/speaker_57
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_33  Cr

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_85  Created 
Successfully created the directory speaker_data_data_same/down/speaker_85
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_4  Created 
Successfully created the directory speaker_data_data_same/down/speaker_4
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_23  Created 
Successfully created the directory speaker_data_data_same/down/speaker_23
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_81  Created 
Successfully created

File copied successfully.
Directory  speaker_data_data_same/down/speaker_100  Created 
Successfully created the directory speaker_data_data_same/down/speaker_100
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_72  Created 
Successfully created the directory speaker_data_data_same/down/speaker_72
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_13  Created 
Successfully created the directory speaker_data_data_same/down/speaker_13
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_40  Created 
Successfully created the directory speaker_data_data_same/down/speaker_40
File copied successfully.
File copied successf

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_43  Created 
Successfully created the directory speaker_data_data_same/down/speaker_43
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_68  Created 
Successfully created the directory speaker_data_data_same/down/speaker_68
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_31  Created 
Successfully created the directory speaker_data_data_same/down/speaker_31
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/down/speaker_110  

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/eight/speaker_57  Created 
Successfully created the directory speaker_data_data_same/eight/speaker_57
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/eight/speaker_33  Created 
Successfully created the directory speaker_data_data_same/eight/speaker_33
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/eight/speaker_36  Created 
Successfully created the directory speaker_data_data_same/eight/speaker_36
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/eight/speaker_53  Created 
Successfully created the directory sp

...

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/yes/speaker_96  Created 
Successfully created the directory speaker_data_data_same/yes/speaker_96
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/yes/speaker_25  Created 
Successfully created the directory speaker_data_data_same/yes/speaker_25
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/yes/speaker_98  Created 
Successfully created the directory speaker_data_data_same/yes/speaker_98
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/yes/speaker_12  Created 
Successfully created the directory speaker_data_dat

Directory  speaker_data_data_same/zero/speaker_119  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_119
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_77  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_77
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_122  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_122
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_78  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_78
File copied successfully.
File copied succes

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_39  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_39
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_71  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_71
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_106  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_106
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_76  Created 
Successfully created the directory speaker

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_34  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_34
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_51  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_51
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_32  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_32
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_38  Created 
Successfully creat

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_12  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_12
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_20  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_20
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_55  Created 
Successfully created the directory speaker_data_data_same/zero/speaker_55
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
Directory  speaker_data_data_same/zero/speaker_121  Created 
Successfully crea

In [137]:
2460/122

20.16393442622951

In [12]:
directory = ["speaker_" + str(x + 1) for x in range(122)]

In [13]:
print(directory)

['speaker_1', 'speaker_2', 'speaker_3', 'speaker_4', 'speaker_5', 'speaker_6', 'speaker_7', 'speaker_8', 'speaker_9', 'speaker_10', 'speaker_11', 'speaker_12', 'speaker_13', 'speaker_14', 'speaker_15', 'speaker_16', 'speaker_17', 'speaker_18', 'speaker_19', 'speaker_20', 'speaker_21', 'speaker_22', 'speaker_23', 'speaker_24', 'speaker_25', 'speaker_26', 'speaker_27', 'speaker_28', 'speaker_29', 'speaker_30', 'speaker_31', 'speaker_32', 'speaker_33', 'speaker_34', 'speaker_35', 'speaker_36', 'speaker_37', 'speaker_38', 'speaker_39', 'speaker_40', 'speaker_41', 'speaker_42', 'speaker_43', 'speaker_44', 'speaker_45', 'speaker_46', 'speaker_47', 'speaker_48', 'speaker_49', 'speaker_50', 'speaker_51', 'speaker_52', 'speaker_53', 'speaker_54', 'speaker_55', 'speaker_56', 'speaker_57', 'speaker_58', 'speaker_59', 'speaker_60', 'speaker_61', 'speaker_62', 'speaker_63', 'speaker_64', 'speaker_65', 'speaker_66', 'speaker_67', 'speaker_68', 'speaker_69', 'speaker_70', 'speaker_71', 'speaker_72', 

In [17]:
wrt = []
x = range(5, 100, 5)
for n in x:
    wrt.append(n) 

In [19]:
wrt.reverse()

In [20]:
wrt

[95, 90, 85, 80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5]